In [ ]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision

In [ ]:
import set_data as sd

# load train set, test set

X_train, Y_train, X_test, Y_test, y_min, y_max, y_mean, y_std = sd.set_data("data/combined_total_data_3")

# print(X_train.shape)
# print(X_train, Y_train)
# print(Y_train[0])

# y_test = Variable(Y_test)
# for i in range(20):
#     print(y_test)

x_train = Variable(X_train)
y_train = Variable(Y_train)


In [ ]:
# regression model

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(28, 128) #input dim = 28, hidden = 128
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.drop2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 16)
        self.drop3 = nn.Dropout(0.1)
        self.fc4 = nn.Linear(16, 1)
        

    def forward(self, x):
        a = F.relu(self.fc1(x))
        a = self.drop1(a)
        a = F.relu(self.fc2(a))
        a = self.drop2(a)
        a = F.relu(self.fc3(a))
        a = self.drop3(a)
        a = self.fc4(a)
        
        return a

In [ ]:
# train

loss_graph = []

In [ ]:
epochs = 50
learningrate = 0.01

model = RegressionModel()
# model.load_state_dict(torch.load('./lenna_d.pth'))
criterion = nn.SmoothL1Loss(size_average = True) 
optimizer = torch.optim.Adam(model.parameters(), lr=learningrate)

print('\nStart Training...\n')

for epoch in range(epochs+1):
    # forward pass
    pred_y = model(x_train.float())
    

    # compute and print loss
    loss = criterion(pred_y.double(), y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    loss_graph.append(loss.item())
    
    if epoch % 10 == 0:
        print ('epoch [{}/{}], loss {}'.format(epoch, epochs, loss.item())) # or loss item
    

In [ ]:
torch.save(model.state_dict(), "./lenna_d.pth")

In [ ]:
fig = plt.figure()
plt.plot(range(epochs+1), loss_graph, label = 'loss', color = 'black')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid(linestyle = '--', color = 'lavender')
plt.show()

In [ ]:
x_test = Variable(X_test)
y_test = Variable(Y_test)

model.eval()
y_pred = model(x_test.float())
after_train = criterion(y_pred.double(), y_test) 
print('Test loss after Training' , after_train.item())

In [ ]:
x_test = Variable(X_test)
y_test = Variable(Y_test)
print(y_test)
criterion = nn.SmoothL1Loss(size_average = True) 

new_model= RegressionModel()
new_model.load_state_dict(torch.load('./lenna_d.pth'))
    
new_model.eval()

y_pred = new_model(x_test.float())
after_train = criterion(y_pred.double(), y_test) 
print('Test loss after Training' , after_train.item())

In [ ]:
def denormalize(data, d_min, d_max, d_mean, d_std):
    
    tmp_data = torch.empty([data.shape[0], data.shape[1]])
    
    for i in range(data.shape[0]):
        e = float(data[i])
        
        #tmp_data[i] = (e*d_std + d_mean)
        tmp_data[i] = (e*d_std + d_mean)*(d_max - d_min) + d_min

    return tmp_data

for i in range(20):
    print(y_test[i], y_pred[i])
# print(float(y_pred[0]))
# print(y_test.float())
# print(y_pred[0])

dy_test = denormalize(y_test, y_min, y_max, y_mean, y_std)
dy_pred = denormalize(y_pred, y_min, y_max, y_mean, y_std)

for i in range(20):
    print(dy_test[i], dy_pred[i])
# print(dy_test)
# print(dy_pred)

